# Homework 1 
## by Gianni Spiga

### Exercise 1 (Empirical risk minimization) (20 pts, 5 pts each)

Consider Poisson model with rate parameter $\lambda$ which has PMF,
$$
p(y|\lambda) = \frac{\lambda^y}{y!} e^{-\lambda},
$$
where $y = 0,1,\ldots$ is some count variable.
In Poison regression, we model $\lambda = e^{\beta^\top x}$ to obtain $p(y | x,\beta)$.

1. Let the loss function for Poisson regression be $\ell_i(\beta) = - \log p(y_i | x_i, \beta)$ for a dataset consisting of predictor variables and count values $\{x_i,y_i\}_{i=1}^n$.  Here $\propto$ means that we disregard any additive terms that are not dependent on $\beta$.  Write an expression for $\ell_i$ and derive its gradient. 

$
\begin{aligned}
\ell_i(\beta) &= - \log p(y_i | x_i, \beta) \\
&= -\log (\frac{(e^{\beta^TX_i})^{y_i}}{y_i!} e^{-e^{\beta^TX_i}})\\
&= -(\beta^Tx_iy_i - \log(y_i!) -e^{\beta^TX_i})\\
&= -\beta^Tx_iy_i + \log(y_i!) + e^{\beta^TX_i}\\
&\propto -\beta^Tx_iy_i + e^{\beta^TX_i}\\
\text{With Gradient}\\
\nabla \ell_i(\beta) &= - x_iy_i +  x_ie^{\beta^TX_i}\\
\end{aligned}
$

2. Show that the empirical risk $R_n(\beta)$ is a convex function of $\beta$.

\nabla \ell_i(\beta) = - x_iy_i +  x_ie^{\beta^TX_i} \\
$$
R_n(\beta) = \frac{1}{n}\sum_{i = 1}^n \ell_i(\theta) = \frac{1}{n}\sum_{i = 1}^n - \log p(y_i | x_i, \beta) \\
\nabla^2 R_n(\beta) = \frac{1}{n}\sum_{i = 1}^n  \nabla^2 (- \log p(y_i | x_i, \beta))\\
\nabla R_n(\beta) = \frac{1}{n}\sum_{i = 1}^n - x_iy_i +  x_ie^{\beta^TX_i} \\
\nabla^2 R_n(\beta) = \frac{1}{n}\sum_{i = 1}^n x_i x_i^Te^{\beta^TX_i} = H  \\
$$

We can then define the Hessian as the following. 

$$
H(\beta) = \begin{bmatrix}
    \frac{\partial^{2} R}{\partial \beta_{1} \partial \beta_{1} } & \ldots & \frac{\partial^{2} R}{\partial \beta_{i} \partial \beta_{p} } \\
    \vdots & \ddots & \vdots \\
     \frac{\partial^{2} R}{\partial \beta_{p} \partial \beta_{1} } & \ldots & \frac{\partial^{2} R}{\partial \beta_{p} \partial \beta_{p} } \\
  \end{bmatrix} \\
  \text{Which we now can define as ...} \\
  H(\beta) = x_i x_i^Te^{\beta^TX_i}
$$

We must show the Hessian matrix is positive definite, we show that for any vector $\mu > 0$:
$$
\begin{aligned}
\mu^TH\mu \geq 0  &= \mu^T x_i x_i^Te^{\beta^TX_i} \mu \\
 &= e^{\beta^TX_i} \mu^T x_i x_i^T \mu \\
 \text{Let} \ a &= \mu^T x_i, b = x_i x_i^T,\{a,b\} \in \mathbb{R} > 0\\
  &= e^{\beta^TX_i} ab  > 0\\
\end{aligned}
$$

Thus the Hessian is positive definite which leads us to conclude that $R_n(\beta)$ is strictly convex. 

3. Consider the mapping $F_\eta(\beta) = \beta - \eta \nabla R_n(\beta)$ which is the iteration of gradient descent ($\eta>0$ is called the learning parameter).  Show that at the minimizer of $R_n$, $\hat \beta$, we have that $F(\hat \beta) = \hat \beta$.

$$
\begin{aligned}
F_\eta(\hat{\beta}) &= \beta - \eta \nabla R_n(\hat{\beta}) \\
&= \beta - 0 \\
&= \beta \\
\end{aligned}
$$

4. I have a script to simulate from this model below.  Implement the gradient descent algorithm above and show that with enough data ($n$ large enough) the estimated $\hat \beta$ approaches the true $\beta$ (you can look at the sum of square error between these two vectors).

In [87]:
import numpy as np

## Simulate from the Poisson regression model (use y,X)
np.random.seed(2022)
n, p = 1000,20
X = np.random.normal(0,1,size = (n,p))
beta = np.random.normal(0,.2,size = (p))
lamb = np.exp(X @ beta)
y = np.random.poisson(lamb)

beta0 = np.random.normal(0.25, .5, size = (p))

def gradDesc(x, y, eta, beta, maxiter = 10000, error = 10e-5):
    i = 1
    n = X.shape[0]
    # Turn into column vectors
    betaNew = beta0.reshape(-1,1)
    betaOld = 0
    y = y.reshape(-1,1)
    print(np.linalg.norm(betaNew - betaOld))
    while i <= maxiter and np.linalg.norm(betaNew - betaOld) > error:
        #print("My i is ", i)
        betaOld = betaNew
        # Resent gradient for each iteration
        Rn = 0

        Rn = (1/n) * (X.T @ np.exp(X @ betaOld) - X.T @ y)
        #print(Rn)
        #for row in range(n): # summation over n
        #    Rn = Rn + -1*X[row,:] * y[row] + X[row,:] * np.exp(betaOld.T @ X[row,:])
        betaNew = betaOld - eta * Rn #gradientRn(Bold)
        #print("Hey this is my beta new " ,betaNew)
        i += 1

    return betaNew

print(beta)

gradDesc(X, y, 0.01,beta0)

[ 0.14159642  0.32078409 -0.25810144 -0.15741574 -0.00722751 -0.34875925
 -0.33806496 -0.28184467 -0.01672773  0.19386419  0.26891229 -0.00396877
  0.08205065 -0.01824143 -0.12716232  0.01491068  0.14876994 -0.0989665
  0.05624015  0.22847606]
2.815558131604145


array([[ 0.18286105],
       [ 0.29255061],
       [-0.23580272],
       [-0.21647555],
       [-0.0246921 ],
       [-0.37453167],
       [-0.3863345 ],
       [-0.28376493],
       [-0.01149554],
       [ 0.15474706],
       [ 0.26400161],
       [-0.02215312],
       [ 0.05063015],
       [-0.03090263],
       [-0.11134687],
       [-0.01535282],
       [ 0.13969634],
       [-0.14053155],
       [ 0.03151566],
       [ 0.19255056]])

# Question 2

Consider the regression setting in which $x_i \in \mathbb R^p$ and $y_i \in \mathbb R$, for $i=1,\ldots,n$ and $p < n$.

1. For a given regressor, let $\hat y_i$ be prediction given $x_i$, and $\hat y$ be the vector form.  Show that linear regression can be written in the form
$$
\hat y = H y,
$$
where $H$ is dependent on $X$ (the matrix of where each row is $x_i$), assuming that $p < n$ and $X$ is full rank.  Give an expression for $H$ or an algorithm for computing $H$.

In linear regression, we have:
$$
\begin{aligned}
\hat{y} &= X\hat{\beta} \\
\hat{y} &= X (X^TX)^{-1}X^Ty \\
\hat{y} &= Hy \\
\text{where } H &= X (X^TX)^{-1}X^T
\end{aligned}
$$

2. Assuming $p < n$ and $X$ is full rank, let $X = U D V^\top$ be the thin singular value decomposition where $U$ is $n \times p$, and $V, D$ is $p \times p$ ($D$ is diagonal). 
    - a) Derive an expression for the OLS coefficients $\hat\beta = A b$ such that $A$ is $p \times p$ and depends on $V$ and $D$, and $b$ is a $p$ vector and does not depend on $D$.   
    - b) Describe a fit method that precomputes these quantities separately
    - c) Use the simulated data $y$ and $X$ in below to find $\hat \beta$ using SVD.
    - d) Call a new data $\tilde X \in \mathbb{R}^{m \times p}$, derive an expression for the predicted $y$ with $\tilde X$ using SVD. 

### a.)
$$
\begin{aligned}
\hat{\beta} &= ((U D V^\top)^TU D V^\top)^{-1}(U D V^\top)^Ty\\
            &= (V D U^\top U D V^\top)^{-1}(U D V^\top)^Ty\\
            &= (V D U^\top U D V^\top)^{-1}(V D U^\top)^Ty\\
            &= (V D D V^\top)^{-1}V D U^\top y\\
            &= (V^\top)^{-1} D^{-1} D^{-1} V^{-1} V D U^\top y\\
            &= V D^{-1} U^\top y\\
            &= Ab \\
            \text{where } A &= V D^{-1}, b= U^\top y, A, \ b \in \mathbb{R}^p

\end{aligned}
$$

### b.) 

One could use numpy's linalg.svd to solve for the matrices $U$,$D$, and $V$ to which they could be then computed to solve for $A$ and $b$.

### c.)

In [110]:
U, D, V = np.linalg.svd(X, full_matrices=False)
A = V @ np.diag(1/D)
b = U.T @ y
betaSVD = A @ b 
print(betaSVD)

[-0.42325615 -0.15765029  0.1264379  -0.22332322  0.40640595 -0.1126941
 -0.17207485 -0.29528352  0.05380933  0.11729673  0.08383002 -0.44454209
  0.12996738 -0.42292366  0.25061563 -0.58863292 -0.29211494 -0.03368431
 -0.37213045  0.02890809]


### d.)

Let the SVD of $\tilde{X}$ be $\tilde{U}, \tilde{D}, \tilde{V}$, then we can define the predicted values of $\hat{y}$ with the following:

$$
\begin{aligned}
\hat{y} &= \tilde{X} \hat{\beta} \\
        &= \tilde{U} \tilde{D} \tilde{V}^T \hat{\beta} \\
        &= \tilde{U} \tilde{D} \tilde{V}^T \tilde{V} \tilde{D}^{-1} \tilde{U}^T y \\
\end{aligned}
$$

3. Consider a regressor that performs OLS using the SVD above, but every instance of D will only use the largest $r$ values on the diagonal, the remainder will be set to 0.  Call this new $p \times p$ matrix $D_r$ ($r < p$).  Then the new coefficient vector is the OLS computed as if the design matrix is modified by $X \gets U D_r V^\top$.  
    - a) Given that you have computed $b$ already, how could you make a method `change_rank` that recomputes $A$ with $D_r$ instead of $D$? 
    - b) Choose $r = 10$, recompute $\hat\beta$ (call it $\hat\beta_{\text{LowRank}}$) in Question 2-c.

### a.)

In our matrix $D$, the entries of singular values are already ordered from largest to smallest, so if we wanted to use the $r$ largest singular values, we'd only have to take the first $r$ entries of the diagonal. This would be easily implementable into a function `change_rank`.

In [149]:
## Simulate from the linear regression model (use y,X)
np.random.seed(2022)
n, p = 100,20
X = np.random.normal(0,1,size = (n,p))
beta = np.random.normal(0,.2,size = (p))
sigma = 1
y = np.random.normal(X @ beta, sigma**2)

In [150]:
def change_rank(X, y, r):
    U,D,V  = np.linalg.svd(X, full_matrices = False)
    p = U.shape[1]
    np.put(D, np.arange(10,20), np.repeat(0, p - r))
    A = V @ np.diag(D)
    b = U.T @ y 
    beta = A @ b
    return beta


### b.)

In [152]:
change_rank(X, y, r = 10)

array([ -1.26673703,  -0.3288066 ,  14.64499225,  -7.47486784,
        23.71705398,  -0.87308367,  29.21796126,   3.17425597,
         1.45353551,  -3.67764043,   6.38765624,   7.89133118,
         5.18706178,   7.05430433, -28.91108728,   1.04644405,
        -2.61613846,  -5.61902513,  -4.51245147,   7.38914609])

# Problem 3

Recall the subset selection problem with tuning parameter $k$,
$$
\min_{\beta : \| \beta \|_0 \le k}\| y - X \beta \|_2^2,
$$
where $\|\beta\|_0 = \#\{j = 1\,\ldots,p : \beta_j \ne 0 \}$.

Notice that we can write this as 
$$
\min_{\beta : |{\rm supp}(\beta)| \le k}\| y - X \beta \|_2^2,
$$
where 
${\rm supp}(\beta) = \{j = 1\,\ldots,p : \beta_j \ne 0 \}$ (${\rm supp}(\beta)$ is the support of $\beta$).

1. Write the subset selection problem in the following form
$$
\min_{S \subseteq \{1,\ldots,p\}, |S|\le k} y^\top P_S y,
$$
where $P_S$ is a projection.  


**Answer**: Currently, we are looking to minimize the residual sum of squares, where we can expand the loss function as: 
$$
\| y - X \beta \|_2^2 = (y - X \beta)^T(y - X \beta)
$$

The classical solution to the least squares problem is the Moore-Penrose psuedo-inverse such that:
$
\hat{\beta} = (X^TX)^{-1}X^Ty
$.
However, we want a projector such that:
$
X\beta = P_S y
$
leading us to now minimize:
$$
\begin{aligned}
\|y-\mathbb P_S y\|_2^2=\|(I-\mathbb P_S)y\|_2^2 \\
= y^T(I-\mathbb P_S)y \\
\end{aligned}
$$
 Where the mimizing projection is $\mathbb P_S = X(X^TX)^{-1}X^T$. We seek to minimize the residual sum of squares by finding the support of $\beta$, the subset which is not mapped to zero in the space $S$. We can use the projection to the orthogonal complement of the subspace. 
$$
\min_{S \subseteq \{1,\ldots,p\}, |S|\le k} y^\top (I-\mathbb P_S) y \\
 = \min_{S \subseteq \{1,\ldots,p\}, |S|\le k} y^\top (I - X(X^TX)^{-1}X^T) y \\
 = \min_{S \subseteq \{1,\ldots,p\}, |S|\le k} y^\top (I - UDV^T(VD^2V^T)^{-1}(UDV^T)^T) y \\
  = \min_{S \subseteq \{1,\ldots,p\}, |S|\le k} y^\top (I - (UD^{-1}V^T)(UDV^T)^T) y \\
$$


2. Suppose that we have a nested sequence of models $S_1\subset S_2 \subset \ldots \subset S_p$ such that $|S_k| = k$ ($|S_k|$ is the cardinality of $S_k$, meaning that it contains $k$ variables).  Prove that $$y^\top P_{S_k} y \ge y^\top P_{S_{k+1}} y$$ for $k=1,\ldots,p-1$.  What does this tell us about the solution to the subset selection problem and the constraint $|S| \le k$?

    (Hint: using the fact that $X^TX$ is positive definite, write $X^TX= VD^2V^T$)

**Answer** 
$$
\begin{aligned}
y^\top P_{S_k} y &\ge y^\top P_{S_{k+1}} y \\
y^\top (I - X(VD^2V^T)^{-1}X^T) y &\ge y^\top (I - \tilde{X}(\tilde{X}^T\tilde{X})^{-1}\tilde{X}^T) y
\end{aligned}
$$


3. Suppose that $X$ is orthogonal, then write a computationally efficient pseudocode to solve the subset selection problem.  Prove that it is correct (your algorithm actually solves subset selection under othogonal design).

4. (Challenge) Suppose that we have that $n = p$ and $y_i = \beta_i + \epsilon_i$ (identity design matrix) where $\epsilon_i$ satisfies 
$$
\mathbb P \left\{ |\epsilon_i| \ge t \right\} \le 2 e^{-t^2 / 2\sigma^2}
$$
for any $t > 0$ (this is true for central Normal RVs) for some $\sigma > 0$.
Suppose that there is some true $S_0 \subset\{1,\ldots,p\}$ such that $|S_0| = k < p$ and ${\rm supp}(\beta) = S_0$.
Prove the following.

__Proposition__
Define $\mu = \min_{j \in S_0} |\beta_j|$ and call $\mu / \sigma$ the signal-to-noise ratio.  Then if 
$$
\frac{\mu}{\sigma} > 2 \sqrt{2 \log \left( \frac{2n}{\delta}\right)},
$$
then the true $S$ is selected by subset selection with probability at least $1 - \delta$.

Hint: rewrite the subset selection problem with $X = I$ and compare the objective at $S_0$ to any other $S$.

**Answer**

$$
\beta_{S_0} = 
\begin{cases}
  \beta_i & \text{if } i \in S_0 \\
  0 & \text{otherwise}
\end{cases}
$$

$\forall$ support set in $S$:
$$
P(\| Y - \beta_{S_0} \|_2^2 \leq \| Y - \beta_{S} \|_2^2 ) \geq 1 - \delta \\
P(\| Y - \beta_{S_0} \|_2^2 - \| Y - \beta_{S} \|_2^2 \leq 0) \geq 1 - \delta \\
P(\| \beta_{S_0} + \epsilon - \beta_S \|_2^2 - \| \epsilon \|_2^2 \leq 0) \geq 1 - \delta \\
P(\| \beta_{S_0} - \beta_S \|_2^2 \| + 2(\beta_{S_0} - \beta_S)^T\epsilon + \| \epsilon \|_2^2 -  \|\epsilon \|_2^2 \geq 0) \geq 1 - \delta \\
P(\| \beta_{S_0} - \beta_S \|_2^2 \| + 2(\beta_{S_0} - \beta_S)^T\epsilon \geq 0) \geq 1 - \delta \\
\mu^2 \leq P(\| \beta_{S_0} - \beta_S \|_2^2 \| + 2(\beta_{S_0} - \beta_S)^T\epsilon \leq 0) \leq \delta \\
$$

# Problem 4

For this exercise, it may be helpful to use the `sklearn.linear_model` module.  I have also included a plotting tool for making the lasso path in ESL.

1. Load the training and test data using the script below.  Fit OLS on the training dataset and compute the test error.  Throughout you do not need to compute an intercept but you should normalize the $X$ (divide by the column norms).

In [156]:
import pickle
with open('hw1.data','rb') as f:
    y_tr,X_tr,y_te,X_te = pickle.load(f)

In [172]:
# Build the OLS model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import Normalizer

X_tr = Normalizer().transform(X_tr)
X_te = Normalizer().transform(X_te)

OLS = LinearRegression().fit(X_tr, y_tr)
OLSpred = OLS.predict(X_te)
mean_squared_error(y_te, OLSpred)

1.0649470897490387

2. Ridge regression: Train and tune ridge regression using a validation set (choose LOOCV) and compute the test error (square error loss).